In [146]:
import requests
from youtube_transcript_api import YouTubeTranscriptApi
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import os
import csv

In [121]:
root_url = "https://www.youtube.com"
# From first to last : Arcane
to_visit = ["/watch?v=bEi-eiFA3OY&persist_gl=1&gl=US"]
visited = []

In [122]:
def find_watch_urls(obj):
  """Recursively search for strings starting with '/watch?v' in a nested JSON object."""
  urls = []
  if isinstance(obj, dict):
    for key, value in obj.items():
      urls.extend(find_watch_urls(value))  # Recurse into dictionary values
  elif isinstance(obj, list):
    for item in obj:
      urls.extend(find_watch_urls(item))  # Recurse into list items
  elif isinstance(obj, str) and re.match(r"^/watch\?v=", obj):  
    urls.append(obj)  # If it's a matching string, add to results
  return urls

In [215]:
def get_new_urls_from_starting_urls(starting_urls):
  new_urls = []
  for url in starting_urls:
    res = requests.get(root_url + url)
    soup = BeautifulSoup(res.text, "html.parser")
    script_tag = soup.find("script", string=re.compile(r"var ytInitialData ="))
    content = script_tag.text
    match = re.search(r'var ytInitialData\s*=\s*({.*?});', content, re.DOTALL)

    if match:
      json_str = match.group(1)
      try:
        json_data = json.loads(json_str)
        #print(json.dumps(json_data, indent=2))  # Pretty-print JSON
      except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
    else:
      print("ytInitialData not found in script text.")

    # Get all next urls
    urls_to_scrape = list(set(find_watch_urls(json_data)))
    urls_to_scrape_full = [s + "&persist_gl=1&gl=US" for s in urls_to_scrape]
    new_urls.extend(urls_to_scrape_full)
    news_urls = list(set(news_urls))
  return new_urls

In [199]:
def crawl_url_from_url(all_urls, to_visit, depth=1): # Parcours en largeur, profondeur de 2, utilisable si on automatise aussi les requêtes à GPT
  if (depth <= 0):
    print("Done!")
    return
  else:
    urls = get_new_urls_from_starting_urls(to_visit)
    filtered_urls = [url for url in urls if url not in all_urls]
    all_urls.extend(filtered_urls)
    crawl_url_from_url(all_urls, to_visit=filtered_urls, depth=depth-1)

In [200]:
#new_urls = get_new_urls_from_starting_urls(to_visit)

In [195]:
# Save in .csv
def save(transcript, input, output_path):
  transcript_d = {"article": transcript, "summary": input}
  new_df = pd.DataFrame([transcript_d])
  file_exists_and_non_empty = os.path.isfile(output_path) and os.path.getsize(output_path) > 0
  new_df.to_csv(output_path, mode='a', header=not file_exists_and_non_empty, index=False, quotechar='"', quoting=csv.QUOTE_MINIMAL)

In [216]:
def get_transcript_save(url, lang=["en"], MAX_WORDS=2500): # or ["en", "fr"]
  match = re.search(r"\/watch\?v=([a-zA-Z0-9_-]+)", url)  
  if match:
    video_id = match.group(1)
    try:
      transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=lang)
      paragraphs = " ".join([t['text'] for t in transcript])
      paragraphs = paragraphs.replace('"', "'")
      paragraphs = " ".join(paragraphs.split()[:MAX_WORDS])
      print(paragraphs + "\n", flush=True)
      user_input = input("Write a summary: ") # Enlever ça et remplacer par un call api pour avoir le résumé
      user_input = user_input.replace('"', "'") 
      if user_input == None:
        user_input = "None"
      save(transcript=paragraphs, input=user_input, output_path="../data/new_ref_data.csv")
      #print(url, " added")
    except:
      print("No transcript found, pass.\n")
      return None
    return paragraphs
  else:
    print("Can't retreive id, pass.\n")
    return None

In [208]:
def retreive_full_save_summary_inputUser(new_urls):
  for url in new_urls:
    get_transcript_save("https://www.youtube.com/watch?v=" + url)

In [ ]:
# Cooking
# to_visit_cook = ["/watch?v=mhDJNfV7hjk"]
# cook_urls = get_new_urls_from_starting_urls(to_visit_cook)
# retreive_full_save_summary_inputUser(cook_urls)

some of the best street food i've ever eaten was in asia every one of those little markets was filled with smells of delicious food that all centered around an asian staple noodles learn to cook those properly and you'll be surprised how quick and easy it is to knock up a delicious dinner first off we're going to soak our noodles now these are rice noodles into a bowl and just pour the hot water over the rice noodles soak and rehydrate 12 to 15 minutes to soften up if you've got a wok perfect if not just a normal size frying pan with big sloping sides pan on get it really nice and hot chicken breast very lean hardly any fat on there you've got this little fillet to slice that off now the secret to getting it really nice and thin and to cut it into strips take your knife keep it nice and flat on the board and sort of slice it in half like a sort of scallop like that we call this butterfly in the chicken take a rolling pin and nice and gently roll over the chicken what it does it sort of

In [210]:
# News
# to_visist_news = ["/watch?v=C7DchvYJxfk"]
# news_urls = get_new_urls_from_starting_urls(to_visist_news)
# print("Found urls: ", len(news_urls))
# retreive_full_save_summary_inputUser(new_urls=news_urls)

Found urls:  23
>> THIS SHOW. >> BEGAN AND CONTINUES BEING THE >> BEGAN AND CONTINUES BEING THE PLACE TO HAVE. PLACE TO HAVE. >> THE. >> THE. >> HARD CONVERSATIONS. >> HARD CONVERSATIONS. >> GOOD MORNING AND. >> GOOD MORNING AND. >> WELCOME TO OUR LIVE. >> WELCOME TO OUR LIVE. >> COVERAGE AS WE COME ON THE >> COVERAGE AS WE COME ON THE AIR WITH BREAKING NEWS AIR WITH BREAKING NEWS OVERNIGHT. OVERNIGHT. >> AN AMERICAN. >> AN AMERICAN. >> AIRLINES PASSENGER FLIGHT >> AIRLINES PASSENGER FLIGHT CRASHED. MID-AIR WITH AN ARMY. CRASHED. MID-AIR WITH AN ARMY. >> HELICOPTER HERE. >> HELICOPTER HERE. >> IN THE WASHINGTON. >> IN THE WASHINGTON. >> DC AREA. >> DC AREA. >> VIDEO CAPTURED. >> VIDEO CAPTURED. >> BY EARTH CAM SHOWS THE MOMENT >> BY EARTH CAM SHOWS THE MOMENT OF COLLISION OVER THE POTOMAC OF COLLISION OVER THE POTOMAC RIVER. IT HAPPENED JUST BEFORE 9 RIVER. IT HAPPENED JUST BEFORE 9 P.M. EASTERN NEAR REAGAN P.M. EASTERN NEAR REAGAN NATIONAL AIRPORT, JUST MINUTES NATIONAL AIRPORT, JUST 

In [211]:
def run_scrape_manual(single_url_watch):
  new_url = get_new_urls_from_starting_urls([single_url_watch])
  print("Found urls: ", len(new_url))
  retreive_full_save_summary_inputUser(new_url)


In [212]:
# AI
# run_scrape_manual("/watch?v=-KK8SuvwoRQ")

Found urls:  22
deep seek is a Chinese AI company and it is shattering the model of American capitalism even as I speak the whole of the American economy is now wedded to Big Tech without big tech there would be very little growth in the US economy without big tech there would be almost no growth at all in US Stock markets with big tech there is both and the myth that the American economy is in good shape is largely based upon the idea that the stock market is at record highs and that's all down to the value of the hype around AI artificial intelligence in other words and yet what we've seen is that a newcomer into the market called Deep seek a Chinese company developed with a tiny amount of money has produced a model of AI that appears to be better than that has being put forward by chat GPT which is the leading model of AI available from US tech companies to people around the world at present deep seek seems to be able to answer questions as well as chat GPT and as well as Microsoft'

In [ ]:
# History
# run_scrape_manual("/watch?v=8qRZcXIODNU")
# run_scrape_manual("/watch?v=QQkmJI63ykI")